In [6]:
# import gc

import torch
import torch.nn.functional as F
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau

import torch_geometric.transforms as T
from torch_geometric.data import NeighborSampler
from torch_geometric.nn import SAGEConv, GCNConv
from torch_geometric import utils, loader
from torch_geometric.utils import subgraph

from torch_geometric.nn import (
    Aggregation,
    MaxAggregation,
    MeanAggregation,
    MultiAggregation,
    SoftmaxAggregation,
    StdAggregation,
    SumAggregation,
    VarAggregation,
    LSTMAggregation
)

# importing obg datatset
from ogb.nodeproppred import PygNodePropPredDataset
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

from pandas.core.common import flatten
# import seaborn as sns
import matplotlib.pyplot as plt
# sns.set(rc={'figure.figsize':(16.7,8.27)})
# sns.set_theme(style="ticks")
from collections import Counter


from tqdm import tqdm
import collections
import os.path as osp
import pandas as pd
import numpy as np
import collections
# from pandas.core.common import flatten
# from scipy.special import softmax

import random

# Setting the seed
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [7]:
opt = 'GCN'

In [9]:
# Load the OGB evaluator for the dataset
evaluator = Evaluator(name='ogbn-products')

# Establish the device for model training 'cuda' if GPU, 'cpu' otherwise
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

# Confirm the device. If it's a GPU, 'cuda' will print
print('Device: {}'.format(device))

Device: cpu


In [10]:
root = osp.join(osp.dirname(osp.realpath('./')), 'content')
if opt == 'GCN':
  dataset = PygNodePropPredDataset( name='ogbn-products', root=root)
else:
  dataset = PygNodePropPredDataset( name='ogbn-products', transform=T.ToSparseTensor(), root=root)

This will download 1.38GB. Will you proceed? (y/N)
 y


Downloaded 1.38 GB: 100%|███████████████████| 1414/1414 [02:29<00:00,  9.45it/s]


Extracting /Users/nipunaupeksha/Desktop/content/products.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.02s/it]


Converting graphs into PyG objects...


100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 327.25it/s]


Saving...


Done!
/Users/nipunaupeksha/miniconda3/envs/msc/lib/python3.8/site-packages/ogb/nodeproppred/dataset_pyg.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.

In [13]:
data = dataset[0]
data

Data(num_nodes=2449029, edge_index=[2, 123718280], x=[2449029, 100], y=[2449029, 1])

In [1]:
# Define the indices of nodes to include in your subset
subset_indices = torch.arange(0, 10000)  # For example, first 10,000 nodes


# Extract the subgraph corresponding to the subset
subset_edge_index, edge_attr, edge_mask = subgraph(subset_indices, data.edge_index, None, relabel_nodes=True, num_nodes=data.num_nodes, return_edge_mask=True)


# Adjust node features and labels for the subset
subset_features = data.x[subset_indices]
subset_labels = data.y[subset_indices]

# Create a new graph object for the subset
subset_graph = data.__class__()
subset_graph.edge_index = subset_edge_index
subset_graph.x = subset_features
subset_graph.y = subset_labels

NameError: name 'torch' is not defined

In [10]:
import numpy as np
from sklearn import metrics as sk
import medmnist
from medmnist import INFO, Evaluator
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


In [13]:
def load_medmnist_data(view, split):
    """
    Load MedMNIST data for the given view and split.

    Args:
        view (str): Either 'c' or 's' indicating the dataset view.
        split (str): The dataset split, one of 'train', 'val', or 'test'.

    Returns:
        (np.ndarray, np.ndarray): Tuple of images and labels.
    """
    info = INFO[f'organ{view}mnist']
    DataClass = getattr(medmnist, info['python_class'])
    transform = transforms.Compose([transforms.ToTensor()])
    dataset = DataClass(split=split, transform=transform, download=True)
    dataloader = DataLoader(dataset, batch_size=len(dataset), shuffle=False)
    for data in dataloader:
        images, labels = data
    images = images.numpy().squeeze()
    labels = labels.numpy().squeeze()
    return images, labels

In [14]:
views = ['c', 's']
num_features = 28 * 28
view = views[0]

train_images, train_labels = load_medmnist_data(view, 'train')
train_data = train_images.reshape(train_images.shape[0], num_features).astype('float')
num_train = train_images.shape[0]

Using downloaded and verified file: /Users/nipunaupeksha/.medmnist/organcmnist.npz


In [17]:
from medmnist import PathMNIST
train_dataset = PathMNIST(split="train", download=True)
train_dataset

100%|███████████████████████| 205615438/205615438 [00:16<00:00, 12097939.83it/s]


Dataset PathMNIST of size 28 (pathmnist)
    Number of datapoints: 89996
    Root location: /Users/nipunaupeksha/.medmnist
    Split: train
    Task: multi-class
    Number of channels: 3
    Meaning of labels: {'0': 'adipose', '1': 'background', '2': 'debris', '3': 'lymphocytes', '4': 'mucus', '5': 'smooth muscle', '6': 'normal colon mucosa', '7': 'cancer-associated stroma', '8': 'colorectal adenocarcinoma epithelium'}
    Number of samples: {'train': 89996, 'val': 10004, 'test': 7180}
    Description: The PathMNIST is based on a prior study for predicting survival from colorectal cancer histology slides, providing a dataset (NCT-CRC-HE-100K) of 100,000 non-overlapping image patches from hematoxylin & eosin stained histological images, and a test dataset (CRC-VAL-HE-7K) of 7,180 image patches from a different clinical center. The dataset is comprised of 9 types of tissues, resulting in a multi-class classification task. We resize the source images of 3×224×224 into 3×28×28, and split 